"librerías"

Metrópolis

In [93]:
function metropolis_1(ΔH, σ, modificador!; β = 1, parametros...)
    n = length(σ)
    i = rand(1:n)
    x = copy(σ[i])
    x = modificador!(x, i, σ)
    dH = ΔH(x,i, σ; parametros...)
    p = exp(-β*dH)
    if rand()< p
        σ[i] = copy(x)
    end
    return σ
end

function metropolis(ΔH, σ, modificador!; N = 1, β = 1,  parametros...)
    for i in 1:N
        σ = metropolis_1(ΔH, σ, modificador!; β = β, parametros...)
    end
    return σ
end


metropolis (generic function with 1 method)

y para el sólido paramagnético tenemos que

In [94]:
H(σ; B = 1, μ = 1) = -B*μ*sum(σ)
ΔH(x, i, σ; B = 1, μ = 1) =-2B*μ*x

ΔH (generic function with 1 method)

In [95]:
function modificador!(x, i, σ) 
    x = -x
end

modificador! (generic function with 1 method)

In [96]:
N = 1000
βs = -10:0.0101:10
M = zeros(length(βs))
E = zeros(length(βs))
i = 0
for β in βs
    i += 1
    σ = rand([-1,1], N )
    σ = metropolis(ΔH, σ, modificador!, N = 10N, β = β, B = 1, μ = 1)
    M[i] = sum(σ)/N
    E[i] = H(σ, B = 1, μ = 1)
end 

y las graficamos como

In [116]:
#plot([1/β for β in βs], M, key = false)
#plot!(xlabel = L"T", ylabel = L"M")
#plot!(xlim = (-100,100))

In [98]:
#plot([1/β for β in βs], E, key = false)
#plot!(xlabel = L"T", ylabel = L"E")
#plot!(xlim = (-100,100))

In [99]:
#plot([β for β in βs], M, key = false)
#plot!(xlabel = L"β", ylabel = L"M")

In [100]:
#plot([β for β in βs], E, key = false)
#plot!(xlabel = L"β", ylabel = L"E")

In [101]:
N = 1000
n = 100
βs = -10:0.0101:10
Mp = zeros(length(βs))
Ep = zeros(length(βs))
σE = zeros(length(βs))
σM = zeros(length(βs))
i = 0
for β in βs
    E = [0.0 for j in 1:1000]
    M = [0.0 for j in 1:1000]
    i += 1
    σ = rand([-1,1], N )
    σ = metropolis(ΔH, σ, modificador!, N = 10N, β = β, B = 1, μ = 1)
    for j in 1:n
        σ = metropolis(ΔH, σ, modificador!, N = 100, β = β, B = 1, μ = 1)
        E[j] = H(σ, B = 1, μ = 1)
        M[j] = sum(σ)/N
    end
    Ep[i] = sum(E)/n
    Mp[i] = sum(M)/n
    σE[i] = sum(E.^2)/n - Ep[i]^2
    σM[i] = sum(M.^2)/n - Mp[i]^2
end 

In [102]:
#plot([1/β for β in βs], Mp, key = false)
#plot!(xlabel = L"T", ylabel = L"\langle M \rangle")
#plot!(xlim = (-100,100))

In [103]:
#plot([1/β for β in βs], Ep, key = false)
#plot!(xlabel = L"T", ylabel = L"\langle E \rangle")
#plot!(xlim = (-100,100))

In [104]:
#plot([1/β for β in βs], σE.*βs.^2, key = false)
#plot!(xlabel = L"T", ylabel = L"C_B")
#plot!(xlim = (-100,100))

In [105]:
#plot([1/β for β in βs], σM.*βs.^2, key = false)
#plot!(xlabel = L"T", ylabel = L"\chi_B")
#plot!(xlim = (-100,100))

In [106]:
N = 900
n = 100
βs = -4:0.04:4
Mp = zeros(length(βs))
Ep = zeros(length(βs))
σE = zeros(length(βs))
σM = zeros(length(βs))
i = 0
for β in βs
    E = [0.0 for j in 1:1000]
    M = [0.0 for j in 1:1000]
    i += 1
    σ = rand([-1,1], N )
    σ = metropolis(ΔH, σ, modificador!, N = 10N, β = β, B = 1, μ = 1)
    σr = reshape(σ, 30,30)
    heatmap(1:30, 1:30, σr, show = :ijulia, title = L"$\beta$ = %$β", aspect_ratio = 1)

end 

LoadError: LoadError: UndefVarError: `@L_str` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
in expression starting at In[106]:16

Ising 2D

In [107]:
function vecinoscuadrados(L)
    filas = [floor(Int,(i-1)/L) for i in 1:L^2]
    [[filas[i]*L+mod1(i-1,L), filas[i]*L+mod1(i+1,L), mod(filas[i]-1, L)*L+mod1(i,L),mod(filas[i]+1, L)*L+mod1(i,L)]
        for i in 1:L^2]
end
vecinoscuadrados(3)

9-element Vector{Vector{Int64}}:
 [3, 2, 7, 4]
 [1, 3, 8, 5]
 [2, 1, 9, 6]
 [6, 5, 1, 7]
 [4, 6, 2, 8]
 [5, 4, 3, 9]
 [9, 8, 4, 1]
 [7, 9, 5, 2]
 [8, 7, 6, 3]

In [108]:
function H(σ; B = 0, J = 1, vecinos = vecinoscuadrados(floor(Int, sqrt(length(σ))))) 
    -B*sum(σ)-0.5*J*sum(σ[j]*σ[i] for i in 1:length(σ) for j in vecinos[i])
end
    
function ΔH(x, i, σ; B = 0, J = 1, vecinos = vecinoscuadrados(floor(Int, sqrt(length(σ))))) 
    -2(B*x+J*x*sum(σ[vecinos[i]]))
end

ΔH (generic function with 1 method)

In [109]:
L = 500
N = L^2
Ts = 2.6:-0.01:0
Mp = zeros(length(Ts))
Ep = zeros(length(Ts))
σE = zeros(length(Ts))
σM = zeros(length(Ts))
i = 0
vecinos = vecinoscuadrados(L)
σ = rand([-1,1], N)
for T in Ts
    E = [0.0 for j in 1:1000]
    M = [0.0 for j in 1:1000]
    i += 1
    σ = metropolis(ΔH, σ, modificador!, N = 10N, β = 1.0 ./T, B = 0.0, J = 1, vecinos = vecinos)
    σr = reshape(σ, L,L)
    heatmap(1:L, 1:L, σr, show = :ijulia, title = L"$T$ = %$T", aspect_ratio = 1)

end 

LoadError: LoadError: UndefVarError: `@L_str` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
in expression starting at In[109]:17

In [110]:
L = 50
N = L^2
vecinos = vecinoscuadrados(L)
n = 1000
Ts = 4:-0.01:0
Mp = zeros(length(Ts))
Ep = zeros(length(Ts))
σE = zeros(length(Ts))
σM = zeros(length(Ts))
i = 0
σ = rand([-1,1], N )
for T in Ts
    @show T
    E = [0.0 for j in 1:n]
    M = [0.0 for j in 1:n]
    i += 1
    σ = metropolis(ΔH, σ, modificador!, N = 500N, β = 1.0 ./T, B = 0.001, J = 1, vecinos = vecinos)
    for j in 1:n
        σ = metropolis(ΔH, σ, modificador!, N = N, β = 1.0 ./T, B = 0.001, J = 1, vecinos = vecinos)
        E[j] = H(σ, B = 0, J = 1, vecinos = vecinos)
        M[j] = sum(σ)/N
    end
    Ep[i] = sum(E)/n
    Mp[i] = sum(M)/n
    σE[i] = sum(E.^2)/n - Ep[i]^2
    σM[i] = sum(M.^2)/n - Mp[i]^2
end 

T = 4.0
T = 3.99
T = 3.98
T = 3.97
T = 3.96
T = 3.95
T = 3.94
T = 3.93
T = 3.92
T = 3.91
T = 3.9
T = 3.89
T = 3.88
T = 3.87
T = 3.86
T = 3.85
T = 3.84
T = 3.83
T = 3.82
T = 3.81
T = 3.8
T = 3.79
T = 3.78
T = 3.77
T = 3.76
T = 3.75
T = 3.74
T = 3.73
T = 3.72
T = 3.71
T = 3.7
T = 3.69
T = 3.68
T = 3.67
T = 3.66
T = 3.65
T = 3.64
T = 3.63
T = 3.62
T = 3.61
T = 3.6
T = 3.59
T = 3.58
T = 3.57
T = 3.56
T = 3.55
T = 3.54
T = 3.53
T = 3.52
T = 3.51
T = 3.5
T = 3.49
T = 3.48
T = 3.47
T = 3.46
T = 3.45
T = 3.44
T = 3.43
T = 3.42
T = 3.41
T = 3.4
T = 3.39
T = 3.38
T = 3.37
T = 3.36
T = 3.35
T = 3.34
T = 3.33
T = 3.32
T = 3.31
T = 3.3
T = 3.29
T = 3.28
T = 3.27
T = 3.26
T = 3.25
T = 3.24
T = 3.23
T = 3.22
T = 3.21
T = 3.2
T = 3.19
T = 3.18
T = 3.17
T = 3.16
T = 3.15
T = 3.14
T = 3.13
T = 3.12
T = 3.11
T = 3.1
T = 3.09
T = 3.08
T = 3.07
T = 3.06
T = 3.05
T = 3.04
T = 3.03
T = 3.02
T = 3.01
T = 3.0
T = 2.99
T = 2.98
T = 2.97
T = 2.96
T = 2.95
T = 2.94
T = 2.93
T = 2.92
T = 2.91
T = 2.9
T = 2.89
T = 

In [111]:
T₀(J) = 2J/(log(1+sqrt(2)))
#plot(Ts, abs.(Mp), key = false)
#plot!(xlabel = L"T", ylabel = L"\langle M \rangle")
#plot!(xlim = (0,4))
#vline!([T₀(1)])

In [112]:
#plot(Ts, Ep, key = false)
#plot!(xlabel = L"T", ylabel = L"\langle E \rangle")
#plot!(xlim = (0,4))
#vline!([T₀(1)])

In [113]:
#plot(Ts, σE./Ts.^2, key = false)
#plot!(xlabel = L"T", ylabel = L"C_v")
#plot!(xlim = (0,4))
#vline!([T₀(1)])

In [114]:
#plot(Ts, σM./Ts.^2, key = false)
#plot!(xlabel = L"T", ylabel = L"\chi")
#plot!(xlim = (0,4))
#vline!([T₀(1)])